This is a notebook which will be used for my capstone project

In [1]:
import pandas as pd
import numpy as np

In [2]:
print("Hello Capstone Project Course!!!!!!!")

Hello Capstone Project Course!!!!!!!


Introduction:

Life School Phuket
I own a small alternative international school in Phuket, Thailand. I am fitness and nature junky and an obsessed surfer. The school has just finished its 5th full year in operation. I started the school to provide an education to my own two boys first and foremost, and they are nearly to the point where they will finish the school and begin higher education online. The school was started with very little capital and provides a community environment for children who might otherwise be home-schooled or a cheaper alternative to very expensive international schools.

The long term plan is to take my children on the road with me and start 4-5 new small schools in strategic locations that are similar to Phuket and have similar challenges for expat families. Rather than compare two cities or neighborhoods within a city, I will compare the neighborhoods of the top surfing spots in the world to attempt to identify the top choices to expand our school brand

Data:

The data I will use for this analysis will be a combination of a list of the top 50 surf spots in the world that was published in a CNN report in 2013. The link to the article is as follows: https://edition.cnn.com/travel/article/50-surf-spots/index.html 

I will also use venue data from Foursquare.

Methodology:

The first step of the analysis will be to web-scrape the CNN website with the article in a Jupyter notebook on IBM Watson and use Beautiful Soup to clean the data. It will then be transformed into a Pandas data frame . Once the data set of the top 50 surf spots is in a data frame format I will need to get the GPS coordinates of each and add them to the data frame.

Finally I will use the Foursquare API to run a k-means clustering exercise to divide the locations into clusters. This will narrow down my selection to the cluster which includes Phuket and the potential next location will be chosen based on its closeness to Phuket.

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

#!pip install pandas
#!pip install requests
!pip install bs4
#!pip install plotly

from bs4 import BeautifulSoup

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-OpenCE

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2021.5.30          |   py37h89c1867_0         141 KB  conda-forge
    geographiclib-1.52         |     pyhd8ed1ab_0          35 KB  conda-forge
    geopy-2.2.0                |     pyhd8ed1ab_0          67 KB  conda-forge
    openssl-1.1.1k             |       h7f98852_0         2.1 MB  conda-forge
    python_abi-3.7             |          2_cp37m           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.4 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.52-pyhd8ed1ab_0
  geopy              conda-forge/noarch::geopy-2.2.0-pyhd8ed1ab_0
  py

Next I will get the data and create a beautiful soup object.

In [2]:
url = "https://edition.cnn.com/travel/article/50-surf-spots/index.html"

data  = requests.get(url).text

In [3]:
soup = BeautifulSoup(data, 'html5lib')

I had to do a bit of googling to figure out how to web scrape just the list of locations from the article but it wasn't too hard.

In [4]:
surfspot_names=soup.find_all("h3")
print(surfspot_names)


[<h3>50. Eisbach, Munich</h3>, <h3>49. Baja Malibu, Baja Norte, Mexico</h3>, <h3>48. Fulong Beach, Taiwan</h3>, <h3>47. Surin Beach, Phuket, Thailand</h3>, <h3>46. Arugam Bay, Sri Lanka</h3>, <h3>45. Surfer's Point, Barbados</h3>, <h3>44. The Bubble, Fuerteventura, Canary Islands</h3>, <h3>43. Black's Beach, San Diego, California</h3>, <h3>42. Cape Hatteras Lighthouse, Buxton, North Carolina</h3>, <h3>41. Surf pool, United Arab Emirates</h3>, <h3>40. Biarritz, France</h3>, <h3>39. Ponta Preta, Maio, Cape Verde</h3>, <h3>38. Tofino, Vancouver Island, Canada</h3>, <h3>37. Byron Bay, New South Wales, Australia</h3>, <h3>36. Les Cavaliers, Anglet, France</h3>, <h3>35. Killer Point, Taghazoute, Morocco</h3>, <h3>34. Pedra Branca Ericeira, Portugal</h3>, <h3>33. Sao Francisco do Sul, Brazil</h3>, <h3>32. Coconuts, Samoa</h3>, <h3>31. Carrapateira, Portugal</h3>, <h3>30. Hanalei Bay, Hawaii</h3>, <h3>29. Rincon, Puerto Rico</h3>, <h3>28. Bundoran Beach, County Donegal, Ireland</h3>, <h3>27. T

In [5]:
surfspot_list = []
for b in surfspot_names[0:]:
    result = b.text.strip()
    surfspot_list.append(result)


In [6]:
print(surfspot_list)

['50. Eisbach, Munich', '49. Baja Malibu, Baja Norte, Mexico', '48. Fulong Beach, Taiwan', '47. Surin Beach, Phuket, Thailand', '46. Arugam Bay, Sri Lanka', "45. Surfer's Point, Barbados", '44. The Bubble, Fuerteventura, Canary Islands', "43. Black's Beach, San Diego, California", '42. Cape Hatteras Lighthouse, Buxton, North Carolina', '41. Surf pool, United Arab Emirates', '40. Biarritz, France', '39. Ponta Preta, Maio, Cape Verde', '38. Tofino, Vancouver Island, Canada', '37. Byron Bay, New South Wales, Australia', '36. Les Cavaliers, Anglet, France', '35. Killer Point, Taghazoute, Morocco', '34. Pedra Branca Ericeira, Portugal', '33. Sao Francisco do Sul, Brazil', '32. Coconuts, Samoa', '31. Carrapateira, Portugal', '30. Hanalei Bay, Hawaii', '29. Rincon, Puerto Rico', '28. Bundoran Beach, County Donegal, Ireland', '27. Tamarindo Beach, Tamarindo, Costa Rica', '26. Pasta Point, Maldives', '25. Backdoor, Oahu, Hawaii', '24. Watergate Bay, Cornwall, England', '23. Cloudbreak, Tavarua 

In the next bunch of cells I split the row data using the period as a delimiter and name the columns, dropping the numbers attached to each location so I end up with a neat list of surf spots.

In [7]:
df = pd. DataFrame(surfspot_list)

In [8]:
df.head()

,0
0,"50. Eisbach, Munich"
1,"49. Baja Malibu, Baja Norte, Mexico"
2,"48. Fulong Beach, Taiwan"
3,"47. Surin Beach, Phuket, Thailand"
4,"46. Arugam Bay, Sri Lanka"


In [9]:
df.columns =['Surf Spot']

In [10]:
df.head()

,Surf Spot
0,"50. Eisbach, Munich"
1,"49. Baja Malibu, Baja Norte, Mexico"
2,"48. Fulong Beach, Taiwan"
3,"47. Surin Beach, Phuket, Thailand"
4,"46. Arugam Bay, Sri Lanka"


In [11]:
new = df["Surf Spot"].str.split(".", n = 1, expand = True)

In [12]:
new.tail()

,0,1
45,5,"P-Pass, Pohnpei, Federated States of Micronesia"
46,4,"Uluwatu and Kuta, Bali, Indonesia"
47,3,"Teahupo'o, Tahiti, French Polynesia"
48,2,"Supertubes, Jeffrey's Bay, South Africa"
49,1,"Pipeline, Oahu, Hawaii"


In [13]:
new.columns =['Drop', 'Surf Spot']

In [14]:
new.head()

,Drop,Surf Spot
0,50,"Eisbach, Munich"
1,49,"Baja Malibu, Baja Norte, Mexico"
2,48,"Fulong Beach, Taiwan"
3,47,"Surin Beach, Phuket, Thailand"
4,46,"Arugam Bay, Sri Lanka"


In [15]:
new.drop(['Drop'], axis = 1)

,Surf Spot
0,"Eisbach, Munich"
1,"Baja Malibu, Baja Norte, Mexico"
2,"Fulong Beach, Taiwan"
3,"Surin Beach, Phuket, Thailand"
4,"Arugam Bay, Sri Lanka"
5,"Surfer's Point, Barbados"
6,"The Bubble, Fuerteventura, Canary Islands"
7,"Black's Beach, San Diego, California"
8,"Cape Hatteras Lighthouse, Buxton, North Carolina"
9,"Surf pool, United Arab Emirates"


Above you see the full list of the surf spots printed. The next step is to get the GPS coordinates of each spot and add them to the data frame.

In [16]:
spot_list = new["Surf Spot"].tolist()
print(spot_list)

[' Eisbach, Munich', ' Baja Malibu, Baja Norte, Mexico', ' Fulong Beach, Taiwan', ' Surin Beach, Phuket, Thailand', ' Arugam Bay, Sri Lanka', " Surfer's Point, Barbados", ' The Bubble, Fuerteventura, Canary Islands', " Black's Beach, San Diego, California", ' Cape Hatteras Lighthouse, Buxton, North Carolina', ' Surf pool, United Arab Emirates', ' Biarritz, France', ' Ponta Preta, Maio, Cape Verde', ' Tofino, Vancouver Island, Canada', ' Byron Bay, New South Wales, Australia', ' Les Cavaliers, Anglet, France', ' Killer Point, Taghazoute, Morocco', ' Pedra Branca Ericeira, Portugal', ' Sao Francisco do Sul, Brazil', ' Coconuts, Samoa', ' Carrapateira, Portugal', ' Hanalei Bay, Hawaii', ' Rincon, Puerto Rico', ' Bundoran Beach, County Donegal, Ireland', ' Tamarindo Beach, Tamarindo, Costa Rica', ' Pasta Point, Maldives', ' Backdoor, Oahu, Hawaii', ' Watergate Bay, Cornwall, England', ' Cloudbreak, Tavarua Island, Fiji', ' Superbank, Gold Coast, Australia', ' Surfrider Beach, Malibu, Calif

I spent a lot of time trying to get the coordinates with geocoder but eventually had to manually get them and uploaded the file to my github.

I uploaded the data to my IBM Watson project and used the insert to code function to create the data frame below

In [17]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_5324950dd2db4d59ac58b27ffb6b49f4 = 'https://s3.us.cloud-object-storage.appdomain.cloud'
else:
    endpoint_5324950dd2db4d59ac58b27ffb6b49f4 = 'https://s3.private.us.cloud-object-storage.appdomain.cloud'

client_5324950dd2db4d59ac58b27ffb6b49f4 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='r3qkgQVfGQJ4IMqihxxM4vIlQVo_EAxVW8BLH4DmlQIt',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_5324950dd2db4d59ac58b27ffb6b49f4)

body = client_5324950dd2db4d59ac58b27ffb6b49f4.get_object(Bucket='capstonecoursefordatascience-donotdelete-pr-ykpu8zksml9tlr',Key='Spots with Geo Data.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


,Surf Spots,Latitude,Longitude
0,"Eisbach, Munich",48.135100,11.582000
1,"Baja Malibu, Baja Norte, Mexico",32.522500,-117.046623
2,"Fulong Beach, Taiwan",25.022000,121.943200
3,"Surin Beach, Phuket, Thailand",7.975473,98.278467
4,"Arugam Bay, Sri Lanka",6.840400,81.836800


In [18]:
df2 = df_data_1

In [19]:
df2 = df2.rename(columns={'Surf Spots':'Surf Spot'})

In [20]:
df2.head()

,Surf Spot,Latitude,Longitude
0,"Eisbach, Munich",48.135100,11.582000
1,"Baja Malibu, Baja Norte, Mexico",32.522500,-117.046623
2,"Fulong Beach, Taiwan",25.022000,121.943200
3,"Surin Beach, Phuket, Thailand",7.975473,98.278467
4,"Arugam Bay, Sri Lanka",6.840400,81.836800


In [21]:
CLIENT_ID = 'J5V3TBLWMG0PLF1IKMBIW0DUFVTNRYXDBWBZRO4KTNS2UGSV' # your Foursquare ID
CLIENT_SECRET = 'UT1DAF5ZQYBN2ZLWE3OTRG1LZNAPUR1QLUXCV42N4JSKNVCJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: J5V3TBLWMG0PLF1IKMBIW0DUFVTNRYXDBWBZRO4KTNS2UGSV
CLIENT_SECRET:UT1DAF5ZQYBN2ZLWE3OTRG1LZNAPUR1QLUXCV42N4JSKNVCJ


In [21]:
address = 'Phuket, Thailand'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Phuket are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Phuket are 7.9366015, 98.3529292.


In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Surf Spot', 
                      'Latitude', 
                      'Longitude', 
                      'Venue', 
                      'Venue Latitude', 
                      'Venue Longitude', 
                      'Venue Category']
    
    return(nearby_venues)
        
                

In [24]:
           
spot_venues = getNearbyVenues(names=df2['Surf Spot'],
                                   latitudes=df2['Latitude'],
                                   longitudes=df2['Longitude']
                                  )

 Eisbach, Munich
 Baja Malibu, Baja Norte, Mexico
 Fulong Beach, Taiwan
 Surin Beach, Phuket, Thailand
 Arugam Bay, Sri Lanka
Surfer's Point, Barbados
 The Bubble, Fuerteventura, Canary Islands
Black's Beach, San Diego, California
 Cape Hatteras Lighthouse, Buxton, North Carolina
 Surf pool, United Arab Emirates
 Biarritz, France
 Ponta Preta, Maio, Cape Verde
 Tofino, Vancouver Island, Canada
 Byron Bay, New South Wales, Australia
 Les Cavaliers, Anglet, France
 Killer Point, Taghazoute, Morocco
 Pedra Branca Ericeira, Portugal
 Sao Francisco do Sul, Brazil
 Coconuts, Samoa
 Carrapateira, Portugal
 Hanalei Bay, Hawaii
 Rincon, Puerto Rico
 Bundoran Beach, County Donegal, Ireland
 Tamarindo Beach, Tamarindo, Costa Rica
 Pasta Point, Maldives
 Backdoor, Oahu, Hawaii
 Watergate Bay, Cornwall, England
 Cloudbreak, Tavarua Island, Fiji
 Superbank, Gold Coast, Australia
 Surfrider Beach, Malibu, California
 The Box, Margaret River Valley, Australia
 Jaws, Maui, Hawaii
 Riyuewan, Sanya, Hain

In [25]:
print(spot_venues.shape)
spot_venues.head()

(294, 7)


,Surf Spot,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Eisbach, Munich",48.1351,11.582,Little London,48.135562,11.580961,Steakhouse
1,"Eisbach, Munich",48.1351,11.582,Globetrotter,48.134611,11.581879,Sporting Goods Shop
2,"Eisbach, Munich",48.1351,11.582,OOH BABY I LIKE IT RAW,48.134023,11.580400,Café
3,"Eisbach, Munich",48.1351,11.582,Mandarin Oriental Munich,48.137257,11.580779,Hotel
4,"Eisbach, Munich",48.1351,11.582,Bar Centrale,48.136821,11.579873,Café


Check how many categories we have obtained.

In [28]:
print('There are {} uniques categories.'.format(len(spot_venues['Venue Category'].unique())))

There are 103 uniques categories.


Analyse each surf spot

In [30]:
# one hot encoding
surf_spots_onehot = pd.get_dummies(spot_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
surf_spots_onehot['Surf Spot'] = spot_venues['Surf Spot'] 

# move neighborhood column to the first column
fixed_columns = [surf_spots_onehot.columns[-1]] + list(surf_spots_onehot.columns[:-1])
surf_spots_onehot = surf_spots_onehot[fixed_columns]

surf_spots_onehot.head()

,Yoga Studio,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Bagel Shop,Bakery,Bank,Bar,Bavarian Restaurant,Beach,Beach Bar,Bed & Breakfast,Beer Garden,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Bus Station,Bus Stop,Café,Campground,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Cupcake Shop,Department Store,Dessert Shop,Egyptian Restaurant,English Restaurant,Falafel Restaurant,Farmers Market,Fish Market,Fountain,Gas Station,German Restaurant,Grocery Store,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Hawaiian Restaurant,Hostel,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Insurance Office,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Lebanese Restaurant,Lounge,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Moroccan Restaurant,Motel,Museum,Music Venue,Nightclub,Noodle House,Optical Shop,Organic Grocery,Other Nightlife,Park,Persian Restaurant,Pharmacy,Pier,Pizza Place,Playground,Plaza,Pool,Pub,Resort,Rest Area,Restaurant,Sandwich Place,Seafood Restaurant,Shop & Service,Snack Place,Spa,Sporting Goods Shop,Steakhouse,Supermarket,Surf Spot,Taco Place,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,"Eisbach, Munich",0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,"Eisbach, Munich",0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Eisbach, Munich",0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Eisbach, Munich",0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Eisbach, Munich",0,0,0,0,0,0,0


Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [31]:
surf_spots_grouped = surf_spots_onehot.groupby('Surf Spot').mean().reset_index()
surf_spots_grouped

,Surf Spot,Yoga Studio,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Bagel Shop,Bakery,Bank,Bar,Bavarian Restaurant,Beach,Beach Bar,Bed & Breakfast,Beer Garden,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Bus Station,Bus Stop,Café,Campground,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Cupcake Shop,Department Store,Dessert Shop,Egyptian Restaurant,English Restaurant,Falafel Restaurant,Farmers Market,Fish Market,Fountain,Gas Station,German Restaurant,Grocery Store,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Hawaiian Restaurant,Hostel,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Insurance Office,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Lebanese Restaurant,Lounge,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Moroccan Restaurant,Motel,Museum,Music Venue,Nightclub,Noodle House,Optical Shop,Organic Grocery,Other Nightlife,Park,Persian Restaurant,Pharmacy,Pier,Pizza Place,Playground,Plaza,Pool,Pub,Resort,Rest Area,Restaurant,Sandwich Place,Seafood Restaurant,Shop & Service,Snack Place,Spa,Sporting Goods Shop,Steakhouse,Supermarket,Taco Place,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Arugam Bay, Sri Lanka",0.00,0.00,0.181818,0.000000,0.000000,0.000000,0.00,0.090909,0.00,0.090909,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.090909,0.000000,0.0,0.000000,0.00,0.0,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.0,0.090909,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.0,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.090909,0.000,0.090909,0.00,0.181818,0.000000,0.090909,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000
1,"Baja Malibu, Baja Norte, Mexico",0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.00,0.0,0.00,0.000000,0.00,0.000000,0.250000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.250000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.00,0.00,0.0,0.000000,0.00,0.00,0.00,0.000000,0.25,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000
2,"Cloud Nine, Siargao Island, Philippines",0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.00,0.0,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.500000,0.000000,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.0,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000,0.500000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000
3,"Eisbach, Munich",0.01,0.01,0.030000,0.000000,0.000000,0.010000,0.01,0.010000,0.07,0.000000,0.01,0.000000,0.010000,0.01,0.01,0.01,0.010000,0.000000,0.000000,0.01,0.010000,0.000000,0.0,0.050000,0.00,0.0,0.01,0.000000,0.03,0.050000,0.010000,0.01,0.01,0.00,0.000000,0.01,0.01,0.01,0.01,0.01,0.0,0.07,0.000000,0.000000,0.000000,0.0,0.01,0.000000,0.050000,0.000000,0.000000,0.02,0.03,0.0,0.02000

Let's print each spot along with the top 5 most common venues

In [32]:
num_top_venues = 5

for hood in surf_spots_grouped['Surf Spot']:
    print("----"+hood+"----")
    temp = surf_spots_grouped[surf_spots_grouped['Surf Spot'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- Arugam Bay, Sri Lanka----
                venue  freq
0    Asian Restaurant  0.18
1  Seafood Restaurant  0.18
2  Italian Restaurant  0.09
3               Beach  0.09
4              Resort  0.09


---- Baja Malibu, Baja Norte, Mexico----
                venue  freq
0  Mexican Restaurant  0.25
1       Grocery Store  0.25
2   Convenience Store  0.25
3                Park  0.25
4           Jazz Club  0.00


---- Cloud Nine, Siargao Island, Philippines----
                       venue  freq
0                      Hotel   0.5
1                 Restaurant   0.5
2                Yoga Studio   0.0
3  Middle Eastern Restaurant   0.0
4            Other Nightlife   0.0


---- Eisbach, Munich----
                 venue  freq
0    German Restaurant  0.07
1  Bavarian Restaurant  0.07
2          Pizza Place  0.05
3          Coffee Shop  0.05
4                Hotel  0.05


---- Fulong Beach, Taiwan----
                       venue  freq
0                      Hotel  0.50
1                      Bea

In [33]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [35]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Surf Spot']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
spot_venues_sorted = pd.DataFrame(columns=columns)
spot_venues_sorted['Surf Spot'] = surf_spots_grouped['Surf Spot']

for ind in np.arange(surf_spots_grouped.shape[0]):
    spot_venues_sorted.iloc[ind, 1:] = return_most_common_venues(surf_spots_grouped.iloc[ind, :], num_top_venues)

spot_venues_sorted.head()

,Surf Spot,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Arugam Bay, Sri Lanka",Seafood Restaurant,Asian Restaurant,Restaurant,Italian Restaurant,Snack Place,Beach,Bar,Resort,Burger Joint,Falafel Restaurant
1,"Baja Malibu, Baja Norte, Mexico",Park,Convenience Store,Grocery Store,Mexican Restaurant,English Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,Cupcake Shop
2,"Cloud Nine, Siargao Island, Philippines",Hotel,Restaurant,Campground,Church,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Cupcake Shop,Department Store
3,"Eisbach, Munich",Bavarian Restaurant,German Restaurant,Hotel,Café,Pizza Place,Coffee Shop,Vietnamese Restaurant,Cocktail Bar,Vegetarian / Vegan Restaurant,Asian Restaurant
4,"Fulong Beach, Taiwan",Hotel,Resort,Beach,Falafel Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Cupcake Shop,Department Store


Cluster into 5 clusters.

In [36]:
# set number of clusters
kclusters = 5

surf_spots_grouped_clustering = surf_spots_grouped.drop('Surf Spot', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(surf_spots_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 0, 4, 1, 4, 3, 1, 1, 1, 1], dtype=int32)

In [37]:
# add clustering labels
spot_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [41]:
surf_spots_merged = df2

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
surf_spots_merged = surf_spots_merged.join(spot_venues_sorted.set_index('Surf Spot'), on='Surf Spot')

surf_spots_merged.head() # check the last columns!

,Surf Spot,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Eisbach, Munich",48.135100,11.582000,1.0,Bavarian Restaurant,German Restaurant,Hotel,Café,Pizza Place,Coffee Shop,Vietnamese Restaurant,Cocktail Bar,Vegetarian / Vegan Restaurant,Asian Restaurant
1,"Baja Malibu, Baja Norte, Mexico",32.522500,-117.046623,0.0,Park,Convenience Store,Grocery Store,Mexican Restaurant,English Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,Cupcake Shop
2,"Fulong Beach, Taiwan",25.022000,121.943200,4.0,Hotel,Resort,Beach,Falafel Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Cupcake Shop,Department Store
3,"Surin Beach, Phuket, Thailand",7.975473,98.278467,1.0,Resort,Thai Restaurant,Hotel,Beach,Bar,Restaurant,Asian Restaurant,Spa,Convenience Store,Nightclub
4,"Arugam Bay, Sri Lanka",6.840400,81.836800,1.0,Seafood Restaurant,Asian Restaurant,Restaurant,Italian Restaurant,Snack Place,Beach,Bar,Resort,Burger Joint,Falafel Restaurant


In [42]:
surf_spots_merged['Cluster Labels']=surf_spots_merged['Cluster Labels'].fillna(0).astype('int')

Since I am interested in similar spots to Phuket, rather than make a map I simply want to create a data frame to see which spots are in the same cluster which I identified as having teh label 1.0.

In [47]:
surf_spots_merged.loc[surf_spots_merged['Cluster Labels'] == 1.0]

,Surf Spot,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Eisbach, Munich",48.135100,11.582000,1,Bavarian Restaurant,German Restaurant,Hotel,Café,Pizza Place,Coffee Shop,Vietnamese Restaurant,Cocktail Bar,Vegetarian / Vegan Restaurant,Asian Restaurant
3,"Surin Beach, Phuket, Thailand",7.975473,98.278467,1,Resort,Thai Restaurant,Hotel,Beach,Bar,Restaurant,Asian Restaurant,Spa,Convenience Store,Nightclub
4,"Arugam Bay, Sri Lanka",6.840400,81.836800,1,Seafood Restaurant,Asian Restaurant,Restaurant,Italian Restaurant,Snack Place,Beach,Bar,Resort,Burger Joint,Falafel Restaurant
9,"Surf pool, United Arab Emirates",25.204800,55.270800,1,Hotel,Lounge,Restaurant,Middle Eastern Restaurant,Japanese Restaurant,Coffee Shop,Bar,Café,Italian Restaurant,Gym / Fitness Center
30,"The Box, Margaret River Valley, Australia",-33.953600,115.073900,1,Café,Supermarket,Korean Restaurant,Grocery Store,Farmers Market,Dessert Shop,Clothing Store,Department Store,Motel,Pub
32,"Riyuewan, Sanya, Hainan Island, China",18.252847,109.511909,1,Park,Asian Restaurant,Hainan Restaurant,Bus Stop,Chinese Restaurant,English Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop
33,"Manu Bay, Raglan, New Zealand",-37.801406,174.871795,1,Café,Board Shop,Ice Cream Shop,Coffee Shop,Supermarket,Pier,Gas Station,Pub,Restaurant,Church
35,"Rincon, Santa Barbara, California",34.335553,-119.407061,1,Beach,Campground,Restaurant,Rest Area,Church,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Cupcake Shop
36,"Montanita Beach, Montanita, Ecuador",-1.826847,-80.752973,1,Nightclub,Hotel,Bar,Restaurant,Hostel,Mediterranean Restaurant,Beach,Pizza Place,Hotel Pool,Seafood Restaurant
38,"Trestles, Orange County, California",33.385000,-117.595000,1,Wings Joint,Farmers Market,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Cupcake Shop,Department Store,Dessert Shop,Egyptian Restaurant


Upon a quick inspection I can further split these into developing and developed markets. My short list of places to consider based upon this exercise has been narrowed down to Puerto Escondido, Montanita, Hainan Island, Arugam Bay. Interestingly Bali did not make the same cluster and my belief is that it is because I had to choose between Kuta and Uluwatu as the article listed both spots which are an hour away from each other. I chose Uluwatu for the wave but Kuta is probably a better location for a school as it is nearest to the main expat community.